# This example shows how fvGP can be used to set up non-stationary kernels 

## To train the GP using the non-staionary kernel via local or hybrid optimizers fvGP accepts the gradient of the kernel as a callable

In [ ]:
import numpy as np
#from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
from fvgp.gp import GP

## Make an example data set

In [ ]:
x = np.linspace(0,1,1000)
def f(x):
    return np.sin(5. * x) + np.cos(10. * x) + (2.* (x-0.4)**2) * np.cos(100. * x)
plt.plot(x,f(x))
x_data = np.random.rand(50)
y_data = f(x_data) + (np.random.rand(len(x_data))-0.5) * 0.5
plt.plot(x,f(x))
plt.scatter(x_data,y_data)
x_data = x_data.reshape(-1,1)

## Define some kernels 

In [ ]:
#stationary
def kernel(x1,x2,hps,obj):
    d = obj._get_distance_matrix(x1,x2)
    #print("eval")
    return hps[0] * np.exp(-d**2/hps[1])
    
#non-stationary and gradient
def nskernel(x1,x2,hps,obj):
    d = obj._get_distance_matrix(x1,x2)
    x0 = np.array([[0.],[0.5],[1.0]])
    w = hps[1:-1]
    l = hps[-1]
    return obj.non_stat_kernel(x1,x2,x0,w,l) * np.exp(-d**2/hps[0])

def nskernel_gradient(x1,x2,hps,obj):
    d = obj._get_distance_matrix(x1,x2)
    x0 = np.array([[0.],[0.5],[1.0]])
    w = hps[1:-1]
    l = hps[-1]
    grad = np.empty((len(hps), len(x1),len(x2)))
    grad[0]   = obj.non_stat_kernel(x1,x2,x0,w,l) * ((d/hps[0])**2) * np.exp(-d**2/hps[0])
    grad[1:]  = obj.non_stat_kernel_gradient(x1,x2,x0,w,l) * np.exp(-d**2/hps[0])
    return grad 

## First the stationary GP 

In [ ]:
my_gp1 = GP(1, x_data,y_data,np.ones((2)), gp_kernel_function=kernel)
my_gp1.train(np.array([[0.001,10.],[0.001,10.]]), method="global")

#let's make a prediction
mean1 = my_gp1.posterior_mean(x.reshape(-1,1))["f(x)"]
var1 =  my_gp1.posterior_covariance(x.reshape(-1,1))["v(x)"]
#print(my_gp1.hyperparameters)
#now we can ask for a new point

In [ ]:
my_gp1.hyperparameters

## Wow the non-stationary GP. We will train globally so the gradient of the kernel is technically not required, but we will check it nontheless 

In [ ]:
my_gp2 = GP(1, x_data, y_data, np.ones((5)), 
            gp_kernel_function=nskernel, gp_kernel_function_grad=nskernel_gradient,\
            ram_economy=False)

my_gp2.train(np.array([[0.001,10.],[-10.,10.],[-10.,10.],[-10.,10.],[0.1,10.]]), method = "global")


#let's make a prediction
mean2 = my_gp2.posterior_mean(x.reshape(-1,1))["f(x)"]
var2 =  my_gp2.posterior_covariance(x.reshape(-1,1))["v(x)"]
#print(my_gp2.hyperparameters)
#now we can ask for a new point

In [ ]:
a,b = my_gp2.test_log_likelihood_gradient(np.ones((5)))

## We'll plot both models, starting eith the stationary. In most cases the non-stationary GP has half the approximation error  

In [ ]:
plt.figure(figsize = (16,10))
plt.plot(x,mean1, label = "posterior mean")
plt.plot(x,f(x), label = "latent function")
plt.fill_between(x, mean1 - 3. * np.sqrt(var1), mean1 + 3. * np.sqrt(var1), alpha = 0.5, color = "grey", label = "var")
plt.scatter(x_data,y_data)
plt.legend()
print("error: ", np.sum(f(x)-mean1)**2)

In [ ]:
plt.figure(figsize = (16,10))
plt.plot(x,mean2, label = "posterior mean", linewidth = 4)
plt.plot(x,f(x), label = "latent function", linewidth = 4)
plt.fill_between(x, mean2 - 3. * np.sqrt(var2), mean2 + 3. * np.sqrt(var2), alpha = 0.5, color = "grey", label = "var")
plt.scatter(x_data,y_data)
plt.legend()
print("error: ", np.sum(f(x)-mean2)**2)